In [1]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)

#os를 import한 이유는 환경변수에 저장된 API 키를 가져오기 위함이다.


In [ ]:
compile = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hi"},],
    temperature=0.0
)
# print(compile.choices[0].message.content)


Hello! How can I assist you today?


### 데이터 전처리 코드

리뷰 JSON 파일 로드

In [2]:
import json

with open('./res/reviews.json', 'r') as f:
    review_list = json.load(f)
# review_list[:3]

# json 파일을 읽어와서 review_list에 저장한다.

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개
- 나쁜 평점 = 별 4개 이하

In [ ]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

# good_cnt, bad_cnt
# review_list에 저장된 데이터를 이용하여 5점과 1점 리뷰의 개수를 센다.

(189, 31)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [ ]:
good_reviews, bad_reviews = [], []
for r in review_list:
    if r['stars'] == 5:
        good_reviews.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        bad_reviews.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

# bad_reviews[:3]
# review_list에 저장된 데이터를 이용하여 5점과 1점 리뷰를 각각 good_reviews와 bad_reviews에 저장한다.

['[REVIEW_START]시설적이 부분에서는 딱히 흠잡을게 없었고 가격대가 좀 있지만 퀄리티가 괜찮은 편이라 so so인데 유독 스파 직원들이 불친절함 표정도 그렇고 말투나 톤이 뭔가 하기 싫다는 듯한 느낌을 받음[REVIEW_END]',
 '[REVIEW_START]직원친절도 최상.  숙소 정비나 시설이 깨끗하고 뒷산뷰도 조용하고 좋았습니다.[REVIEW_END]',
 '[REVIEW_START]깨끗하고 산책하기도 좋아요. 애들이랑 물놀이하기 좋구요. 재미있게 놀다왔습니다.[REVIEW_END]']

In [ ]:
good_reviews_text = '\n'.join(good_reviews)
bad_reviews_text = '\n'.join(bad_reviews)

# gpt prompt에 사용할 수 있도록 good_reviews와 bad_reviews를 하나의 문자열로 만든다.

전처리 함수 작성

In [3]:
import datetime
from dateutil import parser

def preprocess_reviews(path = './res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
    # json 파일을 읽어와서 review_list에 저장한다.

    reviews_good, reviews_bad = [], []
    current_date = datetime.datetime.now()
    date_bound = current_date - datetime.timedelta(days=6*30)
    # 6개월 전 날짜를 구한다. -> 6개월 전 날짜 이후의 리뷰만 사용한다.

    for r in review_list:
        reviews_date_str = r['date']
        try:
            reviews_date = parser.parse(reviews_date_str)
        except(ValueError, TypeError):
            reviews_date = current_date
        # 리뷰의 날짜를 파싱한다. -> 날짜를 파싱할 수 없는 경우 현재 날짜로 처리한다.
        # 크롤링 기준 최신 리뷰의 경우 날짜가 아닌 형태로 표현된다. 해당 경우를 예외처리 하는 코드
        
        if reviews_date < date_bound:
            continue
        # 6개월 이전의 리뷰는 사용하지 않는다.

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        # 6개월 이내의 리뷰를 good_reviews와 bad_reviews에 저장한다.
    
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    # gpt prompt에 사용할 수 있도록 good_reviews와 bad_reviews를 하나의 문자열로 만든다.

    return reviews_good_text, reviews_bad_text

# good, bad = preprocess_reviews()
# print(good[:100])
# print(bad[:100])

### 평가 기준 설정
평가 기준 설정
- MT-Bench 논문 기반 Pairwise Comparision (=LLM 기반 평가)
    - 비교하는 방식 vs. 점수 매기는 방식
    - 점수라는게 애매 할 수 있음 (ex. 어느 정도의 요약 품질이 3점인가?)
    - 경험상 점수보다는 비교가 상대적으로 더 정확한 편

평가 스크립트 작성
- MT-Bench 논문 Prompt에서 일부 단어만 수정 (ex. Korean summarise, accommodation)

In [4]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews] 
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
# 두 개의 답변을 비교하여 어떤 답변이 더 좋은지 판단하는 prompt - MT-Bench 논문 참고

    completion = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion
    # 두 개의 답변을 비교하여 어떤 답변이 더 좋은지에 대한 내용이 담김

### Baseline 모델 개발

In [8]:
PROMPT_BACELINE = f"""아래 숙소 리뷰에 대해 5문장 내로 요약해줘:"""


In [23]:
reviews, _ = preprocess_reviews(path='./res/reviews.json')
# reviesw.json 파일을 읽어와서 6개월 이내의 좋은 리뷰만 reviews에 저장한다.

def summarize(reviews, promt, temperature, model='gpt-3.5-turbo-0125'):
    promt = promt + '\n\n' + reviews

    completion = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': promt}],
        temperature=temperature
    )

    return completion
# review를 요약하는 함수

# print(summarize(reviews, PROMPT_BACELINE).choices[0].message.content)

In [12]:
print(pairwise_eval(reviews, summarize(reviews, PROMPT_BACELINE).choices[0].message.content, summarize(reviews, PROMPT_BACELINE,model= 'gpt-4o-mini').choices[0].message.content).choices[0].message.content)

Both Assistant A and Assistant B provide summaries of the user reviews for the 리솜 리조트, highlighting key aspects such as cleanliness, staff friendliness, and the quality of the spa facilities. However, there are notable differences in the depth and detail of their responses.

Assistant A's summary is concise and focuses on the overall experience, emphasizing the suitability of the resort for families and couples, the cleanliness of the accommodations, and the enjoyable atmosphere. It mentions the winter experience but lacks specific details about potential drawbacks or customer concerns.

On the other hand, Assistant B offers a more comprehensive overview. It not only highlights the positive aspects of the resort, such as cleanliness and staff service, but also addresses some of the negative feedback from customers, including high food prices and the inconvenience of needing to use a cart for transportation within the resort. This inclusion of both positive and negative points provides 

### 대규모 평가 스크립트
- 원래는 많은 수의 다양한 데이터에 대해 평가를 하지만, 동일한 prompt에 대해 temperature 높여서 평가 진행

In [10]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BACELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['리솜포레스트는 즐거운 가족여행을 만들어준 숙소로, 부대시설과 스파가 좋았고 직원 친절도도 좋았다. 또한, 가을에 부모님과 아이들과 함께 방문하기 좋은 편안한 휴양 공간이라고 했다. 방문하기 전후로 리솜포레스트의 온천, 스파, 숙소 상태에 대한 만족을 표현했다.',
 '리솜 리조트는 고객들이 대체로 만족하며, 부대시설이 훌륭하고 숲 속에 위치해 힐링하기에 좋은 곳이라는 리뷰가 많았습니다. 숙소는 깨끗하고 직원들이 친절하게 응대하는 것으로 보여, 다양한 부대시설과 스파를 즐기며 편안한 시간을 보낼 수 있었다는 내용입니다. 또한, 야외 스파를 감상할 수 있는 겨울에 또 방문하고 싶어하는 의견도 있었어요.',
 '버티컬 리솜포레스트의 객실 청결도와 스파 시설로 인해 가족이랑 휴식하기에 최고의 장소로 평가되었어요.훌륭한 경치와 잘 관리된 부대 시설이 호평을 받았으며, 다시 방문 의사가 있습니다.',
 '리솜포레스트는 여러 부대시설이 좋고, 가족이나 친구들과 함께 휴식하기 좋은 숙소로 평가받고 있습니다. 스파 시설이나 부대 서비스에 대한 만족도가 높은 숙소로, 청결하고 편안한 분위기가 인상적이에요. 경치나 산책로도 좋아 이용객들에게 힐링의 시간을 선사합니다. 대체적으로 다양한 이용자들이 만족을 표현하고 있습니다.',
 '1. 리뷰가 긍정적인 요건들을 갖추고 있는 것 같습니다.\n2. 스파 시설과 숙소 청결에 대한 만족도가 높은 것으로 보입니다.\n3. 부대시설과 조식에 대한 다소 비싼 가격을 언급하는 리뷰도 있습니다.\n4. 숙소 내 식당 및 부대시설의 음식가격이 조금 아쉬운 점으로 지적되었습니다.\n5. 산책로 및 숙소에서의 쾌적한 환경이 힐링을 즐기기에 좋은 장소로 언급되었습니다.',
 '리솜포레스트 스파는 깨끗하고 아늑한 숙소로, 부대시설이 좋아서 가족과 힐링하기 좋았습니다. 또한, 직원들의 친절함과 즐거운 시간을 보낼 수 있는 공간으로 평가를 받았습니다.',
 '다양한 부대시설이 좋고, 수영장, 스파부문이 훌륭하며 직원응대도뛰어났음. 인피니티풀까지 완벽한 친절

In [9]:
summary_real_20240526 = '위치가 매우 우수한 숙박시설로, 인사동과 조계사, 경복궁 등 관광지에 도보로 이동할 수 있는 편리한 위치에 있습니다. 객실은 깔끔하며 직원들의 친절한 서비스와 청결한 시설이 인상적입니다. 주변에는 맛집과 편의시설이 많아 편리하며, 교통 접근성도 좋습니다. 전체적으로 만족도가 높고 자주 방문하고 싶은 곳으로 손꼽히는 숙소로 평가됩니다.'
summary_gpt_o1 = '숲속에 위치한 숙소로 자연 속에서 힐링하기에 최적입니다. 객실 상태가 깔끔하고 침구류가 편안해 쾌적한 숙박을 할 수 있습니다. 스파와 수영장 등 다양한 부대시설이 마련되어 있어 가족 단위뿐만 아니라 연인이나 친구들과 함께 즐기기 좋습니다. 직원들이 매우 친절하고 식당들도 품질이 높아 전반적으로 투숙객들에게 긍정적인 경험을 선사합니다. 사계절 내내 아름다운 풍경을 감상할 수 있어 재방문 의사가 높게 나타나는 인기 숙소로 평가됩니다.'

In [10]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, summaries_a, summaries_b):
    a_cnt, b_cnt, draw_cnt = 0, 0, 0
    for i in tqdm(range(len(summaries_a))):
        completion = pairwise_eval(reviews, summaries_a[i], summaries_b[i])
        result = completion.choices[0].message.content
        if '[[A]]' in result:
            a_cnt += 1
        elif '[[B]]' in result:
            b_cnt += 1
        elif '[[C]]' in result:
            draw_cnt += 1 
        else:
            print('Evaluation Error')
    return a_cnt, b_cnt, draw_cnt
# wins, losses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_gpt_o1 for _ in range(len(summaries_baseline))])
# print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

### 모델 고도화1 - 조건들 명시

In [27]:
prompt1 = f"""당신은 요약 전문가 입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 작성되어야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
    2-1. 좋은 예시
        a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
        b) 재방문 예정이라는 평들이 존재합니다.
    2-2. 나쁜 예시
        a) 좋은 숙소였고 방음도 괜찮았습니다.
        b) 재방문 예정입니다.
3. 요약 결과는 최소 4문장, 쵀대 7문장 사이로 작성해주세요.

아래 숙소 리뷰들에 대해 요약해주세요:"""



In [ ]:
eval_count = 10
summaries = [summarize(reviews, prompt1, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_gpt_o1 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

### 모델 고도화2 - 입력 데이터의 품질 증가

In [12]:
def preprocess_reviews2(path = './res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
    # json 파일을 읽어와서 review_list에 저장한다.

    reviews_good, reviews_bad = [], []
    current_date = datetime.datetime.now()
    date_bound = current_date - datetime.timedelta(days=6*30)
    # 6개월 전 날짜를 구한다. -> 6개월 전 날짜 이후의 리뷰만 사용한다.

    for r in review_list:
        reviews_date_str = r['date']
        try:
            reviews_date = parser.parse(reviews_date_str)
        except(ValueError, TypeError):
            reviews_date = current_date
        # 리뷰의 날짜를 파싱한다. -> 날짜를 파싱할 수 없는 경우 현재 날짜로 처리한다.
        # 크롤링 기준 최신 리뷰의 경우 날짜가 아닌 형태로 표현된다. 해당 경우를 예외처리 하는 코드
        
        if reviews_date < date_bound:
            continue
        # 6개월 이전의 리뷰는 사용하지 않는다.

        if len(r['review']) < 30:
            continue
        # 리뷰의 길이가 10자 미만인 경우 사용하지 않음
        # **** 고도화2 추가 ****

        if r['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
        # 6개월 이내의 리뷰를 good_reviews와 bad_reviews에 저장한다.
    
    reviews_good_text = '\n'.join(reviews_good)
    reviews_bad_text = '\n'.join(reviews_bad)
    # gpt prompt에 사용할 수 있도록 good_reviews와 bad_reviews를 하나의 문자열로 만든다.

    return reviews_good_text, reviews_bad_text

reviews, _ = preprocess_reviews2()

In [13]:
eval_count = 10
summaries = [summarize(reviews, prompt1, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_gpt_o1 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:53<00:00,  5.31s/it]

Wins: 5, Losses: 5, Ties: 0


### 모델 고도화3 - Few-Shot Prompting

In [ ]:
reviews_1shot, _ = preprocess_reviews2(path='./res/reviews.json')
pmpt = prompt1+ '\n''\n' + reviews_1shot
summaries_1shot = summarize(reviews_1shot, prompt1, temperature=0.0,model='gpt-4o-2024-11-20').choices[0].message.content

'''
prompt_1shot = f"""당신은 요약 전문가 입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 작성되어야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
    2-1. 좋은 예시
        a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
        b) 재방문 예정이라는 평들이 존재합니다.
    2-2. 나쁜 예시
        a) 좋은 숙소였고 방음도 괜찮았습니다.
        b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summaries_1shot}

아래 숙소 리뷰들에 대해 요약해주세요:"""
'''
prompt_1shot = f"""당신은 요약 전문가 입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summaries_1shot}

아래 숙소 리뷰들에 대해 요약해주세요:"""

In [41]:
eval_count = 5
summaries = [summarize(reviews_1shot, prompt_1shot, temperature=1.0,model='gpt-4-turbo').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_gpt_o1 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 5/5 [00:35<00:00,  7.02s/it]

Wins: 5, Losses: 0, Ties: 0
